In [ ]:
# installing libraries
!pip install transformers
!pip install keras
!pip install datasets
#!pip install --upgrade transformers
#!pip install tf-keras

In [ ]:
# importing libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import transformers
from transformers import XLNetTokenizer, XLNetModel, AdamW, get_linear_schedule_with_warmup

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.utils import resample
from transformers import AutoTokenizer
import joblib
from datasets import Dataset
from datasets import DatasetDict
import tensorflow as tf
from transformers import AutoTokenizer
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

import re
import string
import nltk

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Load the pickled file
oversample_train_new = joblib.load('/content/drive/MyDrive/oversample_trainNEW.pkl')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
ds = Dataset.from_pandas(oversample_train_new)

ds

Dataset({
    features: ['id', 'label', 'tweet', 'length', 'word_count', '__index_level_0__'],
    num_rows: 59440
})

In [ ]:
tra2in_te2st_val2id = ds.train_test_split()

tes2t_val2id = tra2in_te2st_val2id['test'].train_test_split()

tra2in_te2st_val2id_dataset = DatasetDict({
    'train': tra2in_te2st_val2id['train'],
    'test': tes2t_val2id['test'],
    'valid': tes2t_val2id['train']
})

In [ ]:
dat2aset = tra2in_te2st_val2id.remove_columns(['id', 'length', 'word_count', '__index_level_0__'])

In [ ]:
dat2aset

DatasetDict({
    train: Dataset({
        features: ['label', 'tweet'],
        num_rows: 44580
    })
    test: Dataset({
        features: ['label', 'tweet'],
        num_rows: 14860
    })
})

In [ ]:
tokenizer_twitter= AutoTokenizer.from_pretrained("bert-base-cased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
def tokenize_function(oversample_train):
    return tokenizer_twitter(
        oversample_train['tweet'],
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )

token2ized_dataset = dataset.map(tokenize_function, batched=True)

tra2in_dataset = token2ized_dataset['train']
te2st_dataset = token2ized_dataset['test']

Map:   0%|          | 0/44580 [00:00<?, ? examples/s]

Map:   0%|          | 0/14860 [00:00<?, ? examples/s]

In [ ]:
tra2in_dataset

Dataset({
    features: ['label', 'tweet', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 44580
})

In [ ]:
train_set = tra2in_dataset.remove_columns(['tweet']).with_format('tensorflow')

t2f_test_set = tra2in_dataset.remove_columns(['tweet']).with_format('tensorflow')

In [ ]:
train_features = {x: train_set[x] for x in tokenizer_twitter.model_input_names }

train_set_for_final_model = tf.data.Dataset.from_tensor_slices((train_features, train_set['label']))

train_set_for_final_model = train_set_for_final_model.shuffle(len(train_set)).batch(8)

test_features = {x: t2f_test_set[x] for x in tokenizer_twitter.model_input_names}
test_set_for_final_model = tf.data.Dataset.from_tensor_slices((test_features, t2f_test_set["label"]))
test_set_for_final_model =test_set_for_final_model.batch(8)

In [ ]:
model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=3)

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')

print("GPU:", tf.config.list_physical_devices('GPU'))
print("Num GPUs:", len(physical_devices))

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=3)

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=5e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
)


# Train the model
history = model.fit(train_set_for_final_model, epochs=2)

GPU: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Num GPUs: 1


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/2


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
 145/5573 [..............................] - ETA: 47:18 - loss: 0.4998 - sparse_categorical_accuracy: 0.7716

KeyboardInterrupt: 

In [ ]:
from google.colab import drive
import os
import tensorflow as tf

drive.mount('/content/drive')

path = "/content/drive/MyDrive/"

model.save(os.path.join(path, "model"), save_format="tf")


In [ ]:
import pickle
import numpy as np

# Define the file paths
test_set_file = 'test_set_for_final_model.pkl'
tf_test_set_file = 'tf_test_set.pkl'
test_features_file = 'test_features.pkl'

#test_set_np = test_set_for_final_model.numpy()
tf_test_set_np = tf_test_set.numpy()
test_features_np = test_features.numpy()

# Save the variables to files
with open(test_set_file, 'wb') as file:
    pickle.dump(test_set_file, file)

with open(tf_test_set_file, 'wb') as file:
    pickle.dump(tf_test_set_np, file)

with open(test_features_file, 'wb') as file:
    pickle.dump(test_features_np, file)



In [ ]:
predictions = model.predict(test_set_for_final_model)

predicted_labels = tf.argmax(predictions.logits, axis=1).numpy()

true_labels = tf_test_set["label"].numpy()

# Calculate training accuracy
accuracy = sum(predicted_labels == true_labels) / len(true_labels)

print("Accuracy:", accuracy)


In [ ]:
# Get misclassified samples
misclassified_indices_bert = []
for i in range(len(true_labels)):
    if predicted_labels[i] != true_labels[i]:
        misclassified_indices_bert.append(i)

# Print misclassified samples and their predicted labels
for i in misclassified_indices_bert:
    # Convert input_ids back to tokens
    tokens = tokenizer.convert_ids_to_tokens(test_features['input_ids'][i].numpy())
    tweet = " ".join(tokens)
    print(f"Sample {i}: {tweet}")
    print(f"Predicted label: {predicted_labels[i]}")

